In [2]:
import numpy as np
import pandas as pd
import datetime as dt
import sqlite3
path='../raw/'
date_list=['2021-04-19','2021-04-20','2021-04-21','2021-05-05','2021-05-06']

meta=pd.read_csv('../meta/Hwy20_detectors.csv')



In [3]:
for d in date_list:   

    df=pd.read_csv(path+d+'.csv')
    df=df[df['intersection'].isin(meta.Intersection.unique())]
    df['time']=pd.to_datetime(df['time'])
    df=df.rename(columns={'intersection':'signal','code':'event','para':'parameter', 'time':'datetime'})

    df['datetime']=pd.to_datetime(df['datetime'])

    cycle_df=pd.read_csv('../occupancy/'+d+'/'+'cycle_df_'+d+'.csv', index_col=[0])


    cycle_df=cycle_df[cycle_df['cycle_lenght']==130]
    cycle_df=cycle_df.sort_values(['signal','start_cycle']) 
    cycle_df=cycle_df.reset_index(drop=True)



    grouped=cycle_df.groupby(['signal'])
    id_cycle_df=pd.DataFrame([])
    for sets , data in grouped:
        data=data.sort_values(['start_cycle']).reset_index(drop=True)
        data['cycle_id']=data.index
        id_cycle_df=id_cycle_df.append(data)

    id_cycle_df['start_cycle']=pd.to_datetime(id_cycle_df['start_cycle'])
    id_cycle_df['end_cycle']=pd.to_datetime(id_cycle_df['end_cycle'])

    id_cycle_df=id_cycle_df.sort_values(['signal','start_cycle']).reset_index(drop=True)
    id_cycle_df['unique_cycle_id']=id_cycle_df.index

    #cycle_res=cycle_res.reset_index(drop=True)

    cycle_chunk_df=pd.DataFrame([])
    for index, row in id_cycle_df.iterrows():
        print(index)
        if (pd.isnull(row['cycle_lenght'])):
            continue
        else:
            for i in range(0,np.int(row['cycle_lenght']/5)):
                start=(row['start_cycle']+dt.timedelta(seconds=(i)*5))
                end=(row['start_cycle']+dt.timedelta(seconds=(i+1)*5))
                cycle_chunk_df=cycle_chunk_df.append(pd.DataFrame({'cycle_id':row['cycle_id'],'unique_cycle_id':row['unique_cycle_id'],'chunk':i+1,'signal':row['signal'],'chunk_start':start, 'chunk_end':end, 'cycle_start':row['start_cycle'], 'cycle_end':row['end_cycle']}, index=[i]))     


    cycle_chunk_df.to_csv('./'+d+'/'+'cycle_chunk_df'+d+'.csv')
    phases=df[df['event'].isin([1,7,8,9,10,11])]
    phases=phases.reset_index(drop=True)

    phases.to_csv('./'+d+'/'+'phases_1'+d+'.csv')


    #Make the db in memory
    pashe_cycle_db= sqlite3.connect(':memory:')
    #write the tables
    cycle_chunk_df.to_sql('cycle_db', pashe_cycle_db, index=False)
    phases.to_sql('phases_db', pashe_cycle_db, index=False)

    qry_db = '''
        select  *
        from phases_db
           left join cycle_db   using(signal)
            WHERE
            datetime between chunk_start and chunk_end

        '''


    df_query_phase_cycle = pd.read_sql_query(qry_db, pashe_cycle_db)

    df_query_phase_cycle['cycle_start']=pd.to_datetime(df_query_phase_cycle['cycle_start'])
    df_query_phase_cycle['cycle_end']=pd.to_datetime(df_query_phase_cycle['cycle_end'])
    df_query_phase_cycle['cycle_start']=pd.to_datetime(df_query_phase_cycle['cycle_start'])
    df_query_phase_cycle['datetime']=pd.to_datetime(df_query_phase_cycle['datetime'])


    df_query_phase_cycle = df_query_phase_cycle.loc[:,~df_query_phase_cycle.columns.duplicated()]

    df_query_phase_cycle=df_query_phase_cycle.sort_values(['signal','cycle_start','datetime'])


    df_query_phase_cycle.to_csv('./'+d+'/'+'df_query_phase_cycle'+d+'.csv')

    df_query_phase_cycle=df_query_phase_cycle.sort_values(['signal','cycle_start','parameter','datetime','event'])
    # drop duplicates on these columns to have the first event on the end of each chunk 
    # for the next chunk it would be the next event
    df_query_phase_cycle=df_query_phase_cycle.drop_duplicates(['datetime','signal','event','parameter','cycle_id','cycle_start','cycle_end'])
    # one event bellow each record for each signal , cycle id
    df_query_phase_cycle['event_shift_below']=df_query_phase_cycle.groupby(['signal','parameter'])['event'].shift(-1)
    df_query_phase_cycle['datetime_shift_below']=df_query_phase_cycle.groupby(['signal','parameter'])['datetime'].shift(-1)

    df_query_phase_cycle["event_color"] = 'NA'
    df_query_phase_cycle["event_start_time"] = df_query_phase_cycle["datetime"]
    df_query_phase_cycle["event_end_time"] =df_query_phase_cycle["datetime"]

    df_query_phase_cycle['event_color']=np.where((df_query_phase_cycle['event']==1 )& (df_query_phase_cycle['event_shift_below']==7),'Green', df_query_phase_cycle['event_color']  )
    df_query_phase_cycle['event_color']=np.where((df_query_phase_cycle['event']==8) & (df_query_phase_cycle['event_shift_below']==9),'Yellow', df_query_phase_cycle['event_color']  )
    df_query_phase_cycle['event_color']=np.where((df_query_phase_cycle['event']==10) & (df_query_phase_cycle['event_shift_below']==11),'Red', df_query_phase_cycle['event_color']  )

    df_query_phase_cycle['event_end_time']=np.where(df_query_phase_cycle['event_color']!='NA',df_query_phase_cycle['datetime_shift_below'], df_query_phase_cycle['event_end_time']  )
    df_query_phase_cycle['event_start_time']=np.where(df_query_phase_cycle['event_color']!='NA',df_query_phase_cycle['datetime'], df_query_phase_cycle['event_start_time']  )

    df_query_phase_cycle['event_start_time']=np.where(df_query_phase_cycle['event_color']=='NA',np.datetime64('NaT'),df_query_phase_cycle['event_start_time'])
    df_query_phase_cycle['event_end_time']=np.where(df_query_phase_cycle['event_color']=='NA',np.datetime64('NaT'),df_query_phase_cycle['event_end_time'])

    df_query_phase_cycle.to_csv('./'+d+'/'+'df_query_phase_cycle_phaseassigned'+d+'.csv')

    df_query_phase_cycle['below_chunk']=df_query_phase_cycle.groupby(['signal','parameter','unique_cycle_id'])['chunk'].shift(-1)


    df_query_phase_cycle.to_csv('./'+d+'/'+'df_query_phase_cycle_2'+d+'.csv')


    df_query_phase_cycle['below_chunk']=np.where(df_query_phase_cycle['below_chunk'].isna(),26,df_query_phase_cycle['below_chunk'])
    df_query_phase_cycle.to_csv('./'+d+'/'+'df_query_phase_cycle_3'+d+'.csv')


    t=df_query_phase_cycle.dropna(subset=['below_chunk'])
    t=t[t['event_color']!='NA']
    t['chunk']=t['chunk'].astype('int')
    t['l']=t.apply(lambda x: list(np.arange(x['chunk'], x['below_chunk']+1, 1)),axis=1)
    t['index']=t.index

    t.to_csv('./'+d+'/'+'t'+d+'.csv')

    left=t.l.apply(lambda x:pd.Series(x)).stack().reset_index(level=1,drop=True).to_frame('b').reset_index()

    left['location']='NA'
    def location_set(x):
        x['location'].iloc[0]='first'
        x['location'][1:-1]='between'
        x['location'].iloc[-1]='last'
        return(x)

    left=left.groupby('index').apply(location_set)


    left_merged=left.join(t, how='left',on='index', lsuffix='_left', rsuffix='_right')

    left_merged=left_merged[['signal','parameter','cycle_id','unique_cycle_id','event_color','location','b','chunk','below_chunk','event_start_time','event_end_time','cycle_start','cycle_end','event','chunk_start','chunk_end']]

    left_merged.to_csv('./'+d+'/'+'alternative_solution_phase_probability'+d+'.csv')






    left_merged['cycle_start']=pd.to_datetime(left_merged['cycle_start'])
    left_merged['cycle_end']=pd.to_datetime(left_merged['cycle_end'])
    left_merged['event_start_time']=pd.to_datetime(left_merged['event_start_time'])
    left_merged['event_end_time']=pd.to_datetime(left_merged['event_end_time'])
    left_merged['adjust_first_green']='na'
    left_merged['adjust_first_green']=np.where((left_merged['event_end_time']>left_merged['cycle_end']) & (left_merged['location']=='last'),'adjust_next_cycle',left_merged['adjust_first_green'])
    left_merged['adjustment']=left_merged.groupby(['signal','parameter'])['adjust_first_green'].shift(+1)
    left_merged['adjustment_color']=left_merged.groupby(['signal','parameter'])['event_color'].shift(+1)
    left_merged['check_event_end']=left_merged.groupby(['signal','parameter'])['event_end_time'].shift(+1)


    left_merged['adjust_first_green_next_cycle']='na'
    left_merged['adjust_first_green_next_cycle']=np.where((left_merged['adjustment']=='adjust_next_cycle')& (left_merged['check_event_end']>left_merged['cycle_start']) & (left_merged['location']=='first') & (left_merged['b']>1),'adjust_first_green',left_merged['adjust_first_green_next_cycle'])

    left_merged['id']=left_merged.index


    a=left_merged[left_merged['adjust_first_green_next_cycle']=='adjust_first_green']
    a['adjust']=a.apply(lambda x: list(np.arange(1, x['b']+1, 1)),axis=1)
    adjustment_left=a.adjust.apply(lambda x:pd.Series(x)).stack().reset_index(level=1,drop=True).to_frame('chunk_adjustment').reset_index()
    adjustment_left=adjustment_left.rename(columns={'index':'id'})

    adjustment_left_merged=left_merged.merge(adjustment_left, how='outer',on='id')

    adjustment_left_merged.to_csv('./'+d+'/'+'adjustment_left_merged'+d+'.csv')

    adjustment_left_merged['b']=np.where(adjustment_left_merged['adjust_first_green_next_cycle']=='adjust_first_green',adjustment_left_merged.chunk_adjustment,adjustment_left_merged['b'])
    adjustment_left_merged['event_color']=np.where(adjustment_left_merged['adjust_first_green_next_cycle']=='adjust_first_green',adjustment_left_merged.adjustment_color,adjustment_left_merged.event_color)



    adjustment_left_merged=adjustment_left_merged[['signal','parameter','cycle_id','unique_cycle_id','event_color','location','b','chunk','below_chunk','event_start_time','event_end_time','cycle_start','cycle_end','event','chunk_start','chunk_end']]

    adjustment_left_merged.to_csv('./'+d+'/'+'adjustment_left_merged_1'+d+'.csv')










0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

C:\Users\atousaz\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:167: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27